In [3]:
# Import des dépendances
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import pandas as pd
import ast
from random import shuffle

# Charger le modèle français de spaCy
nlp_fr = spacy.load("fr_core_news_md")

# Fonction pour tester la reconnaissance NER sur un texte simple
def test_ner_recognition(text):
    doc = nlp_fr(text)
    spacy.displacy.render(doc, style="ent", jupyter=True)

# Texte simple pour le test de NER
texte_simple = '''Je voudrais aller de Toulouse à Bordeaux.
Comment me rendre à Port-Boulet depuis la gare de Tours ?
Je veux aller voir mon ami Albert à Tours en partant de Bordeaux.'''
test_ner_recognition(texte_simple)

# Chargement du jeu de données à partir d'un fichier CSV pour l'entraînement
def load_dataset(file_path):
    df = pd.read_csv(file_path, sep=',', encoding='utf-8', on_bad_lines='skip')
    return df

# Afficher un aperçu du dataset
df = load_dataset('../../dataset/raw/initial_training_data.csv')
print(df.head())

# Conversion du dataset en format spaCy
def prepare_data_for_spacy(df):
    db = DocBin()
    data = list(zip(df['text'], df['entities']))
    shuffle(data)  # Mélanger les données pour éviter un apprentissage biaisé

    for text, entities in data:
        entities = ast.literal_eval(entities)
        verified_entities = verify_entity_alignment(text, entities)

        if verified_entities:
            doc = nlp_fr.make_doc(text)
            doc.ents = create_entity_spans(doc, verified_entities)
            db.add(doc)
    
    return db

# Vérification de l'alignement des entités
def verify_entity_alignment(text, entities):
    verified_entities = []
    for ent in entities:
        start, end, label = ent['start'], ent['end'], ent['label']
        if text[start:end] == text[start:end]:
            verified_entities.append(ent)
        else:
            print(f"Skipping entity: {text[start:end]} in text: {text}")
    return verified_entities

# Création des spans d'entités
def create_entity_spans(doc, verified_entities):
    ents = []
    for ent in verified_entities:
        start, end, label = ent['start'], ent['end'], ent['label']
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span:
            ents.append(span)
    return ents

# Préparer les données et sauvegarder au format spaCy
db = prepare_data_for_spacy(df)
db.to_disk("../../dataset/processed/processed_training_data.spacy")

# Entrainement du modèle
def train_model(config_path, output_path, training_data_path):
    !python -m spacy init config {config_path} --lang fr --pipeline ner --optimize efficiency --force
    !python -m spacy train {config_path} --output {output_path} --paths.train {training_data_path} --paths.dev {training_data_path}

# Lancement de l'entrainement
train_model('./configuration.cfg', '../../models/saved_models', '../../dataset/processed/processed_training_data.spacy')

# Chargement et test du modèle
def load_and_test_model(model_path, text):
    nlp_model = spacy.load(model_path)
    doc = nlp_model(text)
    return doc

# Définition des couleurs pour les entités
def define_entity_colors():
    return {
        "DEPARTURE": "#ffe899",
        "DESTINATION": "#b1ff5e",
        "ESCALE": "#82b8ff"
    }

# Test du modèle sur différents textes
def test_model_on_texts(model_path):
    texts = [
        '''Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence.
        Je compte me rendre à Bordeaux depuis Marseille pour rendre visite à ma soeur Paris.
        Je dois regarder les trains Toulouse - Brest pour aller voir mon ami Albert.
        Je dois planifier un voyage Nice Toulouse pour les prochaines vacances.
        Une réunion de travail m'oblige à faire Paris - Clermont-Ferrand la semaine prochaine.''',

        '''Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence avec une escale à Nice.
        Je compte me rendre à Bordeaux depuis Marseille en m'arrêtant à Toulouse pour rendre visite à ma soeur Paris.
        Je dois regarder les trains Toulouse - Brest pour aller voir mon ami Albert en faisant une escale à Tours.
        Je dois planifier un voyage Nice Toulouse en passant par Aubervilliers pour les prochaines vacances.
        Une réunion de travail m'oblige à faire Paris - Clermont-Ferrand avec un arrêt par Lyon la semaine prochaine.'''
    ]

    colors = define_entity_colors()
    options = {"ents": list(colors.keys()), "colors": colors}

    for text in texts:
        doc = load_and_test_model("../../models/saved_models/model-best/", text)
        spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

# Exécution des tests sur le modèle
test_model_on_texts("../../models/saved_models/model-best/")

                                                text  \
0          Je voudrais aller de Toulouse à Bordeaux.   
1  Comment me rendre à Port-Boulet depuis la gare...   
2  Je veux aller voir mon ami Albert à Tours en p...   
3         Y a-t-il des trains de Nantes à Montaigu ?   
4  Je souhaite me rendre à Paris en partant de To...   

                                            entities  
0  [{'start': 21, 'end': 29, 'label': 'DEPARTURE'...  
1  [{'start': 20, 'end': 31, 'label': 'DESTINATIO...  
2  [{'start': 36, 'end': 41, 'label': 'DESTINATIO...  
3  [{'start': 23, 'end': 29, 'label': 'DEPARTURE'...  
4  [{'start': 25, 'end': 30, 'label': 'DESTINATIO...  
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: fr
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
